In [84]:
import numpy as np

# global variables
tree_score = None
nodes = {}
clusters = {}
mutations = {}
node_freq = {}
avg_obs_freq = {}

with open('example_output.txt', 'r') as file:
    cur_section = None
    for line in file:
        elements = line.strip().split() 
        if elements == ['TREE_SCORE']:
            cur_section = 'tree_score'
        elif elements == ['PARENT_NODE', 'CHILD_NODE']:
            cur_section = 'node'
        elif elements == ['CLUSTER', 'NODE_OF_CLUSTER']:
            cur_section = 'cluster'
        elif elements == ['MUTATION', 'NODE_OF_MUTATION']:
            cur_section = 'mutation'
        elif elements == ['NODE', 'NODE_FREQS', 'CLADE_FREQS']:
            cur_section = 'node_freq'
        elif elements == ['MUT_ID', 'OBSERVED_FREQS', 'INFERRED_FREQS', 'ABS_OBSERVED_MINUS_INFERRED']:
            cur_section = 'avg_obs_freq'
        elif elements:
            if cur_section == 'tree_score':
                tree_score = elements[0]
            elif cur_section == 'node':
                if int(elements[0]) not in nodes:
                    nodes[int(elements[0])] = set()
                nodes[int(elements[0])].add(int(elements[1]))
            elif cur_section == 'cluster':
                clusters[int(elements[1])] = int(elements[0])
            elif cur_section == 'mutation':
                mutations[elements[0]] = int(elements[1])
            elif cur_section == 'node_freq':
                node_freq[int(elements[0])] = [float(num) for num in elements[1].split(',')]
            elif cur_section == 'avg_obs_freq':
                if mutations[elements[0]] not in avg_obs_freq:
                    avg_obs_freq[mutations[elements[0]]] = []
                avg_obs_freq[mutations[elements[0]]].append([float(num) for num in elements[1].split(',')])

            
    for k, v in avg_obs_freq.items():
        avg_obs_freq[k] = [round(num, 4) for num in list(np.mean(v, axis = 0))]
    
    num_of_mutations = {}
    for _, v in mutations.items():
        if v not in num_of_mutations:
            num_of_mutations[v] = 1
        else:
            num_of_mutations[v] += 1
    print(nodes)

{0: {1}, 1: {2}, 2: {3}, 3: {4}, 4: {5}, 5: {8, 6}, 6: {7}, 8: {9}}


In [86]:
import json

# create the json string

json_string = {}
def create_json(cur, node):
    cur["node"] = node
    cur["cluster"] = clusters[node]
    cur["mutations"] = num_of_mutations[node]
    cur["node_freq"] = node_freq[node]
    cur["avg_obs_freq"] = avg_obs_freq[node]

    if node in nodes and nodes[node]:
        cur["children"] = []
        for i in nodes[node]:
            child = {}
            create_json(child, i)
            cur["children"].append(child)

create_json(json_string, 1)

filename = 'data.json'
with open(filename, "w") as f:
    json.dump(json_string, f, **{"indent": 4})